So I initially started with this draft and decided to leave the copy here. I attempted to calculate screen AOIs to understand if participants were distracted based on their gaze entropy. The key issue with this is that right now the code assumes when the gaze is off the screen AOI (and on the table assembling) it means the participant is distracted. Because we don't have the precise table AOIs, this makes it a bit harder to understand how to interpret gaze entropy as distracted or not. I'm saving this with the hopes that in the future I can train the model more precisely to calculate gaze source (ie high entropy + lots of screen gaze = confusion; high entropy + table gaze = exploration). In our study we should bear in mind that we need to have a ground truth for table AOIs

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from shapely.geometry import Polygon, Point

base_dir = 'gaipat_data/participants'

# list of all participant folders
participants = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# one participant to test
participant_id = participants[0] if participants else None

# made if in case things don't load correctly, could refactor later
if participant_id:
    print(f"[INFO] Testing participant: {participant_id}")
    
    # each participant has figure subfolders: car, house, sc, tb, tc, tsb
    participant_path = os.path.join(base_dir, participant_id)
    figure_names = [d for d in os.listdir(participant_path) if os.path.isdir(os.path.join(participant_path, d))]
    
    if figure_names:
        figure_name = figure_names[0]  # just pick the first figure for testing
        print(f"[INFO] Testing figure: {figure_name}")
        
        # build file paths
        screen_path = os.path.join(participant_path, figure_name, 'screen', 'gazepoints.csv')
        table_path = os.path.join(participant_path, figure_name, 'table', 'gazepoints.csv')
        
        if not os.path.exists(screen_path):
            print(f"[ERROR] Missing screen gaze file: {screen_path}")
        elif not os.path.exists(table_path):
            print(f"[ERROR] Missing table gaze file: {table_path}")
        else:
            # Load and display shape + column info
            df_screen = pd.read_csv(screen_path)
            df_table = pd.read_csv(table_path)
            
            print(f"[SUCCESS] Loaded screen gaze: {df_screen.shape}")
            print(f"[SUCCESS] Loaded table gaze: {df_table.shape}")
            print(f"[INFO] Screen columns: {df_screen.columns.tolist()}")
            print(f"[INFO] Table columns: {df_table.columns.tolist()}")
    else:
        print(f"[ERROR] No figure folders found for {participant_id}")
else:
    print("[ERROR] No participant folders found.")


[INFO] Testing participant: 87891249
[INFO] Testing figure: sc
[SUCCESS] Loaded screen gaze: (4525, 3)
[SUCCESS] Loaded table gaze: (3432, 3)
[INFO] Screen columns: ['timestamp', 'x', 'y']
[INFO] Table columns: ['timestamp', 'x', 'y']


In [2]:
def merge_preprocess_gaze(df_screen, df_table):
    """
    Merges and preprocesses gaze data from screen and table into a single dataframe.
    - add source column
    - drop NaN
    - convert to seconds
    - sort by timestamp
    """

    # assign source labels
    df_screen['source'] = 'screen'
    df_table['source'] = 'table'

    # drop NaN, specifying columns in abundance of caution
    df_screen = df_screen.dropna(subset=['x', 'y', 'timestamp'])
    df_table = df_table.dropna(subset=['x', 'y', 'timestamp'])

    # merge dataframes
    df = pd.concat([df_screen, df_table], ignore_index=True)

    # sort by timestamp
    df = df.sort_values('timestamp').reset_index(drop=True)

    return df

df_merged = merge_preprocess_gaze(df_screen, df_table)

Reusable functions to calculate gaze entropy and gaze entropy over time

In [3]:
def calc_gaze_entropy(xy_points, bins=10):
    """
    Calculate spatial entropy of gaze data.
    Shannon entropy quantifies how unpredictable the location of a point is based on its x and y values

    Parameters:
        xy_points (np.ndarray): 2D array of shape (N, 2) for gaze coordinates
        bins (int): number of bins per axis for histogram
    Returns:
        float: Shannon entropy in bits
    """
    if xy_points.shape[0] < 2:
        return np.nan  # when there's not enough data to compute entropy

    # 2D histogram over gaze space
    H, _, _ = np.histogram2d(xy_points[:, 0], xy_points[:, 1], bins=bins)

    # Flatten and normalize to get probabilities
    p = H.flatten() / np.sum(H)
    # remove zero bins to avoid log(0)
    p = p[p > 0]  

    # compute Shannon entropy in bits
    return entropy(p, base=2)

# calculating entropy in 2 second chunks (2000 ms) with 500 ms steps
def compute_entropy_over_time(df, window_size=2000, step_size=500, bins=10):
    """
    Slides a time window over gaze data and computes spatial entropy per window.
    Returns a DataFrame with: start_time, end_time, entropy
    helps understand how gaze patterns change over time
    """
    results = []
    start_time = df['timestamp'].min()
    end_time = df['timestamp'].max()
    current = start_time

    while current + window_size <= end_time:
        window = df[(df['timestamp'] >= current) & (df['timestamp'] < current + window_size)]
        if len(window) >= 2:
            xy = window[['x', 'y']].to_numpy()
            ent = calc_gaze_entropy(xy, bins=bins)
            results.append({
                'start_time': current,
                'end_time': current + window_size,
                'entropy': ent
            })
        current += step_size

    return pd.DataFrame(results)

entropy_df = compute_entropy_over_time(df_merged)

# check it worked
print(entropy_df.head())

      start_time       end_time   entropy
0  1706261126399  1706261128399  3.489089
1  1706261126899  1706261128899  3.360848
2  1706261127399  1706261129399  2.654981
3  1706261127899  1706261129899  2.052651
4  1706261128399  1706261130399  2.584189


Setting thresholds, 3 distraction levels (could make it more in the future if we have more physiological signals)

In [4]:
# split into quartiles, we want 3 attention categories: low, medium, high
thresholds = entropy_df['entropy'].quantile([1/3, 2/3]).values

# Bin entropy values into distraction levels
entropy_df['distraction_level'] = pd.cut(
    entropy_df['entropy'],
    bins=[-np.inf, thresholds[0], thresholds[1], np.inf],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

# check if it worked
print(entropy_df.head())

      start_time       end_time   entropy  distraction_level
0  1706261126399  1706261128399  3.489089                  2
1  1706261126899  1706261128899  3.360848                  1
2  1706261127399  1706261129399  2.654981                  0
3  1706261127899  1706261129899  2.052651                  0
4  1706261128399  1706261130399  2.584189                  0


Helper functions for AOI & gaze-tagging

In [12]:
# to get Areas of Interest (AOIs) from slides_{figure}.csv --> tell us slide number, title, block, destination, figure, and arrow
def extract_slide_aois(slides_df, aoi_names=None):
    """
    Extract AOIs from slides_{figure}.csv as Shapely polygons.
    Returns a dictionary: {slide_id: {aoi_name: Polygon}}
    """
    if aoi_names is None:
        aoi_names = sorted(set(col.split('_')[0] for col in slides_df.columns if '_x0' in col))

    slide_aois = {}

    for _, row in slides_df.iterrows():
        slide_id = int(row['id'])
        slide_aois[slide_id] = {}

        for name in aoi_names:
            try:
                coords = [
                    (float(row[f'{name}_x0']), float(row[f'{name}_y0'])),
                    (float(row[f'{name}_x1']), float(row[f'{name}_y1'])),
                    (float(row[f'{name}_x2']), float(row[f'{name}_y2'])),
                    (float(row[f'{name}_x3']), float(row[f'{name}_y3']))
                ]
                # Check for NaNs or infinities
                if not np.all(np.isfinite(coords)):
                    continue

                coords.append(coords[0])  # close the polygon
                poly = Polygon(coords)

                if poly.is_valid and not poly.is_empty:
                    slide_aois[slide_id][name] = poly

            except (KeyError, TypeError, ValueError):
                continue  # skip invalid AOIs

    return slide_aois


# test if it works
slides_tb = pd.read_csv("gaipat_data/setup/slides_house.csv")
slide_aois = extract_slide_aois(slides_tb)

# Inspect AOIs for slide 0
slide_aois[0]


{'released': <POLYGON ((0.396 0.436, 0.538 0.436, 0.538 0.8, 0.396 0.8, 0.396 0.436))>,
 'title': <POLYGON ((0 0, 1 0, 1 0.181, 0 0.181, 0 0))>}

In [13]:
def tag_on_target_gaze(df_screen, screen_states, slide_aois):
    """
    Tags each gaze point as on-target if it falls inside any AOI polygon.
    Requires: slide_aois = {slide_id: {aoi_name: Polygon}}
    Returns: df_screen with 'on_target' column
    """

    # Merge slide ID into gaze data using timestamp
    df = pd.merge_asof(
        df_screen.sort_values('timestamp'),
        screen_states.sort_values('timestamp'),
        on='timestamp',
        direction='backward'
    )

    def is_on_target(row):
        slide_id = row['slide']
        if slide_id not in slide_aois:
            return False
        point = Point(row['x'], row['y'])
        for poly in slide_aois[slide_id].values():
            if poly.contains(point):
                return True
        return False

    df['on_target'] = df.apply(is_on_target, axis=1)
    return df

# example to test it works
screen_states = pd.read_csv("gaipat_data/participants/5530740/house/screen/states.csv")
df_screen_tagged = tag_on_target_gaze(df_screen, screen_states, slide_aois)

print(df_screen_tagged.head())  

       timestamp   x   y  source  slide  on_target
0  1706261126399 NaN NaN  screen      9      False
1  1706261126410 NaN NaN  screen      9      False
2  1706261126420 NaN NaN  screen      9      False
3  1706261126432 NaN NaN  screen      9      False
4  1706261126443 NaN NaN  screen      9      False


Calculate on target ratio: for each entropy window (e.g. 2 seconds), how much of the gaze was on-target (inside the AOIs defined for that task slide.)

In [ ]:
def compute_on_target_ratio(entropy_df, df_screen_tagged):
    """
    Add on-target gaze ratio to each entropy window.
    
    Parameters:
        entropy_df: DataFrame with ['start_time', 'end_time', ...]
        df_screen_tagged: screen gaze points with ['timestamp', 'on_target']

    Returns:
        entropy_df with added 'on_target_ratio' column
    """
    ratios = []
    for _, row in entropy_df.iterrows():
        start, end = row['start_time'], row['end_time']
        window = df_screen_tagged[
            (df_screen_tagged['timestamp'] >= start) &
            (df_screen_tagged['timestamp'] < end)
        ]

        if len(window) == 0:
            ratios.append(np.nan)
        else:
            ratio = window['on_target'].sum() / len(window)
            ratios.append(ratio)

    entropy_df['on_target_ratio'] = ratios
    return entropy_df

# check if it works
print("[DEBUG] Entropy time range:", entropy_df['start_time'].min(), "→", entropy_df['end_time'].max())
print("[DEBUG] Gaze time range:", df_screen_tagged['timestamp'].min(), "→", df_screen_tagged['timestamp'].max())

print(df_screen_tagged['on_target'].value_counts(dropna=False))

print("Slide AOI keys (from slides_*.csv):", list(slide_aois.keys())[:10])
print("Slide IDs in tagged gaze:", df_screen_tagged['slide'].dropna().unique())


entropy_df = compute_on_target_ratio(entropy_df, df_screen_tagged)

# check to see it worked, should have diff distraction levels
print(entropy_df.head())
print(entropy_df.columns)

[DEBUG] Entropy time range: 1706261126399 → 1706261181899
[DEBUG] Gaze time range: 1706261126399 → 1706261182072
on_target
False    3543
True      982
Name: count, dtype: int64
Slide AOI keys (from slides_*.csv): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Slide IDs in tagged gaze: [9]
      start_time       end_time   entropy  distraction_level  on_target_ratio
0  1706261126399  1706261128399  3.489089                  2         0.524390
1  1706261126899  1706261128899  3.360848                  1         0.530864
2  1706261127399  1706261129399  2.654981                  0         0.447205
3  1706261127899  1706261129899  2.052651                  0         0.443750
4  1706261128399  1706261130399  2.584189                  0         0.500000
Index(['start_time', 'end_time', 'entropy', 'distraction_level',
       'on_target_ratio'],
      dtype='object')


Note to self for later to improve model performance:
- add pupil info for better means of interpreting cognitive state via pupil dilation
- add AOIs for both screen and table (need to calculate from slides provided)
- Add events.csv logic to: Track task steps, extract errors or redundant actions, add task_phase, action_count, or task_efficiency features